In [34]:
import tensorflow as tf

In [35]:

tf.test.is_gpu_available()


True

In [36]:

import pandas as pd



# Load the dataset
file_path = 'Truth_Seeker_Model_Dataset.csv'
file_path2 = 'Features_For_Traditional_ML_Techniques.csv'
file_path3 = 'Downloads/Truth_Seeker_Model_Dataset_With_TimeStamps 1.xlsx'
df = pd.read_csv(file_path)

df2 = pd.read_csv(file_path2)

df3 = pd.read_excel(file_path3)

df = df.drop(columns=["Unnamed: 0"])

df2 = df2.drop(columns=["Unnamed: 0"])

df3 = df3.drop(columns=["Unnamed: 0"])


In [37]:

df = df.sample(frac=1, random_state=42).reset_index(drop=True)


In [38]:


import warnings
warnings.filterwarnings("ignore")



In [39]:


from transformers import *  # this is HuggingFace library
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel, BertConfig




In [40]:

# this will download the BERT Tokenizer
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")  
# this will download the BERT Trained Model
# output_hidden_states=False, as we are training & not interested in output state.
config = BertConfig.from_pretrained("bert-base-uncased",output_hidden_states=False) # dropout=0.2, attention_dropout=0.2
bert_model = TFBertModel.from_pretrained('bert-base-uncased', config=config)



loading file vocab.txt from cache at C:\Users\T2330114\.cache\huggingface\hub\models--bert-base-uncased\snapshots\86b5e0934494bd15c9632b12f734a8a67f723594\vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\T2330114\.cache\huggingface\hub\models--bert-base-uncased\snapshots\86b5e0934494bd15c9632b12f734a8a67f723594\tokenizer_config.json
loading file tokenizer.json from cache at C:\Users\T2330114\.cache\huggingface\hub\models--bert-base-uncased\snapshots\86b5e0934494bd15c9632b12f734a8a67f723594\tokenizer.json
loading file chat_template.jinja from cache at None
loading configuration file config.json from cache at C:\Users\T2330114\.cache\huggingface\hub\models--bert-base-uncased\snapshots\86b5e0934494bd15c9632b12f734a8a67f723594\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_pro

In [41]:


import re
from tqdm import tqdm


def clean_text(temp):
    temp=re.sub("@\S+", " ", temp)
    temp=re.sub("https*\S+", " ", temp)
    temp=re.sub("#\S+", " ", temp)
    temp=re.sub("\'\w+", '', temp)
    temp=re.sub(r'\w*\d+\w*', '', temp)
    temp=re.sub('\s{2,}', " ", temp)
   
    return temp.strip()

df['tweet_clean'] = df['tweet'].apply(clean_text)
df['stat'] = df['statement'].apply(clean_text)
df['auth'] = df['author'].apply(clean_text)
df['mank'] = df['manual_keywords'].apply(clean_text)
df['fivlab'] = df['5_label_majority_answer'].apply(clean_text)
df['thrlab'] = df['3_label_majority_answer'].apply(clean_text)
#sentences = df['tweet_clean'] + " "+ df['stat'] + " " + df['auth'] + " " + df['mank'] + " " + df['fivlab'] + " " + df['thrlab'] 

sentences = df['tweet_clean'] + " "+ df['stat'] + " " + df['auth'] + " " + df['mank']


In [42]:



import numpy as np


input_ids=[]
attention_masks=[]

for sent in tqdm(sentences):
    bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,truncation=True,max_length =28,pad_to_max_length = True,return_attention_mask = True)
    input_ids.append(bert_inp['input_ids'])
    attention_masks.append(bert_inp['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
target = np.array(df['BinaryNumTarget'])

100%|█████████████████████████████████████████████████████████████████████████| 134198/134198 [03:12<00:00, 697.71it/s]


In [43]:

bert_tokenizer.convert_ids_to_tokens(101)



'[CLS]'

In [44]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test,train_mask,test_mask=train_test_split(input_ids,target,attention_masks,test_size=0.2,random_state=42)

In [45]:


bert_model.summary()

Model: "tf_bert_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
Total params: 109,482,240
Trainable params: 109,482,240
Non-trainable params: 0
_________________________________________________________________


In [46]:


from tensorflow.keras.layers import Input, Embedding, SpatialDropout1D, LSTM, Dense, Dropout, Flatten, Concatenate, GlobalMaxPool1D , Bidirectional
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
import numpy as np

# Textual Input
# text_input = Input(shape=(max_sequence_length,), name='text_input')
# embedding = Embedding(input_dim=max_num_words, output_dim=embedding_dim, input_length=max_sequence_length)(text_input)
input_ids_in = tf.keras.layers.Input(shape=(28,), name='input_token', dtype='int32')
input_masks_in = tf.keras.layers.Input(shape=(28,), name='masked_token', dtype='int32') 
embedding_layer = bert_model(input_ids_in, attention_mask=input_masks_in)[0] 
lstm = Bidirectional(LSTM(8, return_sequences=True, dropout=0.1, recurrent_dropout=0))(embedding_layer)
pooling = GlobalMaxPool1D()(lstm)
X = Dense(8, activation='relu')(pooling)
X = Dropout(0.2)(X)
X = Dense(1, activation='sigmoid')(X)

model = Model(inputs=[input_ids_in, input_masks_in], outputs=X)


for layer in bert_model.layers[:3]:  
    layer.trainable = False

model.summary()




Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_token (InputLayer)       [(None, 28)]         0           []                               
                                                                                                  
 masked_token (InputLayer)      [(None, 28)]         0           []                               
                                                                                                  
 tf_bert_model_1 (TFBertModel)  TFBaseModelOutputWi  109482240   ['input_token[0][0]',            
                                thPoolingAndCrossAt               'masked_token[0][0]']           
                                tentions(last_hidde                                               
                                n_state=(None, 28,                                          

In [47]:
import os

checkpoint_path = "finaltraining_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)


model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])



In [48]:

history=model.fit([X_train,train_mask],y_train,batch_size=32,epochs=8,validation_data=([X_test,test_mask],y_test),callbacks=[cp_callback])

Epoch 1/8
3355/3355 [==============================] - ETA: 0s - loss: 0.3798 - accuracy: 0.8271
Epoch 1: saving model to finaltraining_1\cp.ckpt
3355/3355 [==============================] - 217s 61ms/step - loss: 0.3798 - accuracy: 0.8271 - val_loss: 0.2495 - val_accuracy: 0.9007
Epoch 2/8
3354/3355 [============================>.] - ETA: 0s - loss: 0.2751 - accuracy: 0.8937
Epoch 2: saving model to finaltraining_1\cp.ckpt
3355/3355 [==============================] - 205s 61ms/step - loss: 0.2751 - accuracy: 0.8937 - val_loss: 0.2284 - val_accuracy: 0.9058
Epoch 3/8
3355/3355 [==============================] - ETA: 0s - loss: 0.2425 - accuracy: 0.9060
Epoch 3: saving model to finaltraining_1\cp.ckpt
3355/3355 [==============================] - 201s 60ms/step - loss: 0.2425 - accuracy: 0.9060 - val_loss: 0.1967 - val_accuracy: 0.9190
Epoch 4/8
3354/3355 [============================>.] - ETA: 0s - loss: 0.2245 - accuracy: 0.9118
Epoch 4: saving model to finaltraining_1\cp.ckpt
3355/335

In [49]:
os.listdir(checkpoint_dir)

['.ipynb_checkpoints',
 'checkpoint',
 'cp.ckpt.data-00000-of-00001',
 'cp.ckpt.index']

In [50]:


# Loads the weights
model.load_weights(checkpoint_path)


In [51]:
loss, acc = model.evaluate([X_test,test_mask],y_test, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

839/839 - 32s - loss: 0.1799 - accuracy: 0.9281 - 32s/epoch - 38ms/step
Restored model, accuracy: 92.81%


In [52]:



from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict on the test set
y_pred_prob = model.predict([X_test,test_mask])



839/839 [==============================] - 38s 41ms/step


In [53]:

y_pred = (y_pred_prob > 0.5).astype(int)

# Calculate accuracy, precision, and recall
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")
print(f"F1-score: {f1}")

Test Accuracy: 0.9281296572280179
Test Precision: 0.9250485996112031
Test Recall: 0.9353523587652883
F1-score: 0.9301719457013574


In [54]:



predict_sent = sentences[:20]

input_ids2=[]
attention_masks2=[]


for sent in tqdm(predict_sent):
    bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,truncation=True,max_length =28,pad_to_max_length = True,return_attention_mask = True)
    input_ids2.append(bert_inp['input_ids'])
    attention_masks2.append(bert_inp['attention_mask'])

input_ids_predict=np.asarray(input_ids2)
attention_masks_predict=np.array(attention_masks2)




100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 573.53it/s]


In [55]:

preddf1 = pd.read_csv('ISOT_Fake.csv') ## Fake

preddf1['label'] = 0.0

preddf2 = pd.read_csv('ISOT_True.csv') ##Real

preddf2['label'] = 1.0

preddf1.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0.0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0.0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0.0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0.0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0.0


In [56]:
preddf2.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1.0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1.0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1.0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1.0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1.0


In [57]:

combine = pd.concat([preddf1, preddf2], ignore_index=True)

combine = combine.sample(frac=1, random_state=42).reset_index(drop=True)

combine.head()

,title,text,subject,date,label
0,Ben Stein Calls Out 9th Circuit Court: Committ...,"21st Century Wire says Ben Stein, reputable pr...",US_News,"February 13, 2017",0.0
1,Trump drops Steve Bannon from National Securit...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 5, 2017",1.0
2,Puerto Rico expects U.S. to lift Jones Act shi...,(Reuters) - Puerto Rico Governor Ricardo Rosse...,politicsNews,"September 27, 2017",1.0
3,OOPS: Trump Just Accidentally Confirmed He Le...,"On Monday, Donald Trump once again embarrassed...",News,"May 22, 2017",0.0
4,Donald Trump heads for Scotland to reopen a go...,"GLASGOW, Scotland (Reuters) - Most U.S. presid...",politicsNews,"June 24, 2016",1.0


In [58]:


def clean_text(temp):
    temp=re.sub("@\S+", " ", temp)
    temp=re.sub("https*\S+", " ", temp)
    temp=re.sub("#\S+", " ", temp)
    temp=re.sub("\'\w+", '', temp)
    temp=re.sub(r'\w*\d+\w*', '', temp)
    temp=re.sub('\s{2,}', " ", temp)
    return temp.strip()


combine['tile'] = combine['title'].apply(clean_text)
combine['text_clean'] = combine['text'].apply(clean_text)
combine['sub'] = combine['subject'].apply(clean_text)
combine['dat'] = combine['date'].apply(clean_text)


#sentences3 = combine['tile'] + " " + combine['text_clean'] + " " + combine['sub'] + " " + combine['dat']
sentences3 = combine['tile'] + " " + combine['text_clean'] + " " + combine['sub']



In [59]:

input_ids3=[]
attention_masks3=[]


for sent in tqdm(sentences3):
    bert_inp=bert_tokenizer.encode_plus(sent,add_special_tokens = True,truncation=True,max_length =28,pad_to_max_length = True,return_attention_mask = True)
    input_ids3.append(bert_inp['input_ids'])
    attention_masks3.append(bert_inp['attention_mask'])

new_input_ids=np.asarray(input_ids3)
new_attention_mask=np.array(attention_masks3)
new_labels = np.array(combine['label'])



100%|████████████████████████████████████████████████████████████████████████████| 44898/44898 [07:35<00:00, 98.57it/s]


In [60]:

X_trainnew, X_testnew, y_trainnew, y_testnew,train_masknew,test_masknew=train_test_split(new_input_ids,new_labels,new_attention_mask,test_size=0.2,random_state=42)

In [61]:

checkpoint_path2 = "finaltraining_2/cp.ckpt"
checkpoint_dir2 = os.path.dirname(checkpoint_path2)

# Create a callback that saves the model's weights
cp_callback2 = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path2,
                                                 save_weights_only=True,
                                                 verbose=1)



from tensorflow.keras.optimizers import Adam

# Recompile the model with a smaller learning rate
model.compile(
    optimizer=Adam(learning_rate=2e-5),  # Smaller learning rate for fine-tuning
    loss='binary_crossentropy',         # Same loss function as before
    metrics=['accuracy']                # Same evaluation metrics
)




In [62]:

# Fine-tuning with new data
history2 = model.fit(
    [X_trainnew, train_masknew],  # New input data
     y_trainnew,                           # New labels
    batch_size=32,
    epochs=7,  # Use fewer epochs for fine-tuning
    validation_data=([X_testnew, test_masknew], y_testnew),callbacks=[cp_callback2] 
)

Epoch 1/7
1122/1123 [============================>.] - ETA: 0s - loss: 0.8376 - accuracy: 0.6241
Epoch 1: saving model to finaltraining_2\cp.ckpt
1123/1123 [==============================] - 85s 64ms/step - loss: 0.8375 - accuracy: 0.6241 - val_loss: 0.6144 - val_accuracy: 0.7145
Epoch 2/7
1123/1123 [==============================] - ETA: 0s - loss: 0.5101 - accuracy: 0.7718
Epoch 2: saving model to finaltraining_2\cp.ckpt
1123/1123 [==============================] - 70s 62ms/step - loss: 0.5101 - accuracy: 0.7718 - val_loss: 0.4062 - val_accuracy: 0.8274
Epoch 3/7
1123/1123 [==============================] - ETA: 0s - loss: 0.3416 - accuracy: 0.8614
Epoch 3: saving model to finaltraining_2\cp.ckpt
1123/1123 [==============================] - 68s 60ms/step - loss: 0.3416 - accuracy: 0.8614 - val_loss: 0.2304 - val_accuracy: 0.9117
Epoch 4/7
1122/1123 [============================>.] - ETA: 0s - loss: 0.2053 - accuracy: 0.9268
Epoch 4: saving model to finaltraining_2\cp.ckpt
1123/1123 [

In [63]:


# Loads the weights
model.load_weights(checkpoint_path2)



In [64]:


loss, acc = model.evaluate([X_testnew, test_masknew], y_testnew, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))




281/281 - 11s - loss: 0.0507 - accuracy: 0.9821 - 11s/epoch - 39ms/step
Restored model, accuracy: 98.21%


In [65]:


# Predict on the test set
y_pred_prob2 = model.predict([X_testnew, test_masknew])

281/281 [==============================] - 15s 42ms/step


In [66]:

y_pred2 = (y_pred_prob2 > 0.5).astype(int)

# Calculate accuracy, precision, and recall
accuracy2 = accuracy_score(y_testnew, y_pred2)
precision2 = precision_score(y_testnew, y_pred2)
recall2 = recall_score(y_testnew, y_pred2)
f12 = f1_score(y_testnew, y_pred2)

print(f"Test Accuracy: {accuracy2}")
print(f"Test Precision: {precision2}")
print(f"Test Recall: {recall2}")
print(f"F1-score: {f12}")

Test Accuracy: 0.9820712694877506
Test Precision: 0.9745899745899745
Test Recall: 0.988056206088993
F1-score: 0.9812768926619374
